In [1]:
import os

import cv2
import numpy as np
import torch
import torch.nn.functional as F

from PIL import Image
from model import build_segmenter
from utils.dataset import RefDataset
import utils.config as config
from utils.dataset import tokenize

/home/wayve/saurabh/CRIS.pytorch/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_dir = "/dev/shm/saurabh/refcocog_g/CRIS_R101/"
args = config.load_cfg_from_cfg_file("config/refcocog_g/cris_r101.yaml")
model, _ = build_segmenter(args)
model = torch.nn.DataParallel(model).cuda()
model = model.eval()

args.model_dir = os.path.join(model_dir, "best_model.pth")
if os.path.isfile(args.model_dir):
    checkpoint = torch.load(args.model_dir)
    model.load_state_dict(checkpoint['state_dict'], strict=True)
else:
    raise ValueError(
        "=> resume failed! no checkpoint found at '{}'. Please check args.resume again!"
        .format(args.model_dir))


2022-09-14 21:08:43.179 | INFO     | model:build_segmenter:41 - Backbone with decay=478, Head=124


In [ ]:
image_input_dir = "orange_tennis.jpeg"
image_input = cv2.imread(image_input_dir)
image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
rfd = RefDataset(None, None, None, None, None, input_size=args["input_size"], word_length=args["word_len"])
image_tensor = rfd.convert_image_to_tensor(image_input)
image_tensor = torch.unsqueeze(image_tensor, dim=0)


In [ ]:
args

In [ ]:

Image.fromarray(image_input)

In [ ]:
#query = "Both men in front of the cake"
query = "Random"
sentence_tokens = tokenize(query, 22, True)
sentence_tokens = sentence_tokens.cuda(non_blocking=True)
pred = model(image_tensor, sentence_tokens)
pred = torch.sigmoid(pred)

if pred.shape[-2:] != image_tensor.shape[-2:]:
    pred = F.interpolate(pred,
                         size=image_tensor.shape[-2:],
                         mode='bicubic',
                         align_corners=True).squeeze()


pred = pred.cpu().numpy()
pred = np.array(pred > 0.35)
pred = np.array(pred*255, dtype=np.uint8)
h, w, _ = image_input.shape
mat, mat_inv = rfd.getTransformMat(image_input.shape[:2], True)
pred = cv2.warpAffine(pred, mat_inv, (w, h),
                      flags=cv2.INTER_CUBIC,
                      borderValue=0.)


color = np.array([255, 0, 0])
pred = np.expand_dims(pred, axis=2)
mask = np.full_like(image_input, color)
pred = np.concatenate((pred, pred, pred), axis=2)
mask *= pred


Image.fromarray(np.uint8(0.7*image_input + 0.3*pred))
